In [ ]:
#Uncomment for when Chrome version changes

#from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
##HEB

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime
import json

#Driver settings

options = webdriver.ChromeOptions()
#options.add_argument('headless')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
 

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')


#To get around javascript barriers in HEB website, there is a seperate account for each metro area where the zipcode is 
#preset for that account. HEB uses extensive bot prevention measures, and each login requires passing several CAPTCHA 
#tests, but even with those extra steps the data collection is much faster. Short forms of cities are used in the 
#lookup below for the email addresses and the long forms are used in the final output.

mlookup={"angelo":"San Angelo","htx":"Houston","sa":"San Antonio","atx":"Austin","cc":"Corpus Christi","colleges":"College Station","waco":"Waco","victoria":"Victoria","mcallen":"McAllen"}

def hebscrape(market):
    login=market+"price31415@gmail.com"
    pw=market+"price31415!!"
    
    driver = webdriver.Chrome(executable_path=r'./chromedriver.exe', options=options)

    driver.get("https://www.heb.com/my-account/login?originalURL=https%3A%2F%2Fwww.heb.com%2Fmy-account%2Fdash-board")
    
    driver.find_element_by_xpath('//*[@id="email"]').send_keys(login)
    driver.find_element_by_xpath('//*[@id="password"]').send_keys(pw)
    
    driver.find_element_by_xpath('//*[@id="loginForm"]/form/div[4]/div/button/span').click()
    WebDriverWait(driver, 600).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="root"]/header/div[1]/div[2]/div[1]/form/div/input')))
    
    #Rather than utilizing search bar, you can go to the webaddress associated with that search query
                       
    brizand=("Karbach","Shiner","Deep Ellum","Blue Moon","Saint Arnold","Corona","Michelob Ultra","Truly","White Claw")
    summary=list()
    
    
    for b in brizand:
        suburl="https://www.heb.com/search/?q="+b+"+beer"
        driver.get(suburl)
        time.sleep(7)
               
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        
        #Some cities have different versions of the HEB website, so there are different tags
        #for the tiles that have the product information. If the first doesn't work then the second
        #one will.

        if len(soup.find_all(class_='sc-ztn198-0 jQudBd sc-cltr8j-0 fNTXmj'))>0:
            product= soup.find_all(class_='sc-ztn198-0 jQudBd sc-cltr8j-0 fNTXmj')            
            
            price=[]
            pkg= []
            brand=[]
            for i in product:
                try:

                    title=str(re.findall(rf'aria-label=\"{b}.+?each\.',str(i))[0])
                    title=title[12:]
                    title=title.split(',')
                    
                    brand.append(title[0])
                    price.append(title[1][:-5].strip())
                    
                    
                    title2=str(re.findall(rf'alt=\"{b}.+?\"',str(i))[0])
                    title2=title2.split(',')
                    pkg.append(title2[1][1:])
                except:
                    continue


        else:
            price_selector = soup.find_all(class_='cat-price-number')
            product= soup.find_all(class_='responsivegriditem product-grid-large-fifth product-grid-small-6')

            if len(product)==0:
                product= soup.find_all(class_='ztn198-0 gIcCcD cltr8j-0 jfvaNx')


            price=[]
            for i in price_selector:
                price.append(str(re.findall(r'\$[0-9.]*',str(i))[0]))

            pkg= []
            brand=[]
            for i in product:
                L=len(str(re.findall(rf'aria-label=\"{b}.*[^\$]',str(i))))
                brand.append(str(re.findall(rf'aria-label=\"{b}.*[^\$]',str(i)))[14:L-6])
                L=len(str(re.findall(rf'aria-label=\"{b}.*\n*\t*\t*.......',str(i))))
                tes=str(re.findall(rf'aria-label=\"{b}.*\n*\t*\t*.......',str(i)))[L-8:L-2]

                if tes[0]!='t':
                    pkg.append(str(re.findall(rf'aria-label=\"{b}.*\n*\t*\t*.......',str(i)))[L-8:L-2])
                else:
                    pkg.append(str(re.findall(rf'aria-label=\"{b}.*\n*\t*\t*.......',str(i)))[L-7:L-2])

                   
        #Make sure that there are an equal number of prices, brand names, and product types for each brand
        
        print([len(pkg),len(brand),len(price)])
        
        for i in range(len(pkg)):
            summary.append({"cdate":Current_Date,"store":"HEB","market":mlookup[market],"pkg":pkg[i],"brand":brand[i],"price":price[i]})

    driver.close()
    return summary
         

cities=["angelo","htx","sa","atx","cc","colleges","waco","victoria","mcallen"]

Combined = list()



for i in cities:
    Combined.append(hebscrape(i))


filename=Current_Date+"heb.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()
print("done")



In [ ]:
##Total Wine

import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime

options = webdriver.ChromeOptions()

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')




def twscrape(market):
    
    def hoverclick(xxpath):
        hover = ActionChains(driver).move_to_element(driver.find_element_by_xpath(xxpath))
        action = webdriver.common.action_chains.ActionChains(driver)

        tries=0
        while tries<3:
            try:
                hover.perform()
                break
            except:
                tries+=1
        time.sleep(2)
        
        action.click()
        action.perform()
    
    url="https://www.totalwine.com/store-finder/search?q="+market.replace(" ","%20")+"&radius=15"
    driver.get(url)
    time.sleep(6)
    
    try:
        hoverclick('/html/body/div[9]/div[2]/div/button')
        time.sleep(2)
    except:
        time.sleep(2)
    
    c=0
    
    while c<10:
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        c+=1
    time.sleep(1)
    while c<10:
        ActionChains(driver).send_keys(Keys.PAGE_UP).perform()
        c+=1
    time.sleep(1)
    
    if market !="San Antonio":

        targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[1]/div[2]/section/div[2]/section[2]/ol/li[1]/div[2]/div[2]/a')
        targ.click()
        time.sleep(2)
    else:
        targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[1]/div[2]/section/div[2]/section[2]/ol/li[2]/div[2]/div[2]/a')
        targ.click()
        time.sleep(2)
    
    try:
        targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[2]/div/div[2]/div/div[2]/button')
        targ.click()
    except:
        try:
            driver.find_element_by_xpath('/html/body/div[9]/div[2]/div/div[1]/button/img').click()
            targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[2]/div/div[2]/div/div[2]/button')
            targ.click()
        except:
            driver.find_element_by_xpath('/html/body/div[10]/div[2]/div/div[1]/button/img').click()
            targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[2]/div/div[2]/div/div[2]/button')
            targ.click()
        
    summary=list()    
    

    brizand=("karbach","shiner","deep%20ellum","blue%20moon","saint%20arnold","corona","michelob%20ultra","truly","white%20claw")
    
    
    
    for b in brizand:
        suburl="https://www.totalwine.com/search/all?text="+b+"&pageSize=72&department=Beer"
        driver.get(suburl)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')


        price_selector = soup.find_all(class_='price__1JvDDp_x')
        product= soup.find_all(class_='productCard__2nWxIKmi')

        price=[]
        for i in price_selector:
            price.append(str(re.findall(r'\$[0-9.]*',str(i))[0]))


        brand=[]
        for i in product:
            brand.append(str(re.findall(r'true">[A-Za-z \-]*[^\<]',str(i))[0])[6:])

        pkg= []
        for i in product:
                L=len(str(re.findall(r'<span>.*?\<',str(i))[0]))
                pkg.append(str(re.findall(r'<span>.*?\<',str(i))[0][6:L-1])) 

        print([len(pkg),len(brand),len(price)])
        
        
        
        
        for i in range(len(pkg)):
            summary.append({"cdate":Current_Date,"store":"Total Wine","market":market,"pkg":pkg[i],"brand":brand[i],"price":price[i]})
        
    return summary








driver = webdriver.Chrome(executable_path=r'./chromedriver.exe', options=options)


cities=["San Antonio","Austin","Dallas","Houston","Corpus Christi"]

Combined = list()

for i in cities:
    Combined.append(twscrape(i))
     
driver.close()

filename=Current_Date+"tw.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()


print("done")

In [ ]:
##Kroger
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime
import json

options = webdriver.ChromeOptions()
#options.add_argument('headless')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
 

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')



def krogerscrape(market):
    login=market+"31415@gmail.com"
    pw=market+"31415!!"
    
    driver = webdriver.Chrome(executable_path=r'./chromedriver.exe', options=options)

    driver.get("https://www.kroger.com/signin?redirectUrl=/")
    
    time.sleep(4)
    
    driver.find_element_by_xpath('//*[@id="SignIn-emailInput"]').send_keys(login)
    driver.find_element_by_xpath('//*[@id="SignIn-passwordInput"]').send_keys(pw)
    driver.find_element_by_xpath('//*[@id="SignIn-submitButton"]').click()
    
    
    
    time.sleep(8)
                   

    brizand=("karbach","shiner","deep%20ellum","blue%20moon","saint%20arnold","corona","michelob%20ultra","truly","white%20claw")
        
    summary=list()
    
    for b in brizand:
        suburl="https://www.kroger.com/search?query="+b+"%20beer&searchType=default_search&fulfillment=all"
        driver.get(suburl)
        #WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[2]/div[2]/div[3]/div/div/div/div[1]')))
        try:
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[2]/div[2]/div[3]/div/div/div/div[1]')))
        except:
            time.sleep(10)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        
        product= soup.find_all(class_='AutoGrid-cell min-w-0')
        price_selector= soup.find_all(typeof="Price")    
        
                
        price=[]
        for i in price_selector:
            
            L=len(str(re.findall(r'value="[0-9.].+meta',str(i))[0]))
            price.append(str(re.findall(r'value="[0-9.].+meta',str(i))[0])[7:L-7])
        
        pkg= []
        brand=[]
        for i in product:
            try:
                L=len(str(re.findall(r'description".*?-',str(i))[0]))
                brand.append(str(re.findall(r'description".*?-',str(i))[0])[13:L-2])
                L=len(str(re.findall(r'description".*?-',str(i))[0]))+1
                pkg.append(str(re.findall(r'description".*?h3',str(i))[0])[L:-4])

            except:
                brand.append('NA')
                pkg.append('NA')
                continue
                
        print([len(pkg),len(brand),len(price)])
        

        
        for i in range(len(price)):
            summary.append({"cdate":Current_Date,"store":"Kroger","market":market.capitalize(),"pkg":pkg[i],"brand":brand[i],"price":price[i]})
        
    driver.close()
    return summary
           

#cities=["San Angelo","Houston","San Antonio","Austin","Corpus Christi","College Station","Waco","Victoria","Mcallen"]
cities=["houston","Dallas"]

Combined = list()


for i in cities:
    Combined.append(krogerscrape(i))
    


filename=Current_Date+"kroger.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()
print("done")

In [ ]:
##Albertsons Randalls Tom Thumb

import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime

options = webdriver.ChromeOptions()

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')


mlookup={"Dallas":"75001","Houston":"77001"}


def randyscrape(market):
    
    def clickit(xxpath):
            tries=0
            while tries<3:
                try:
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xxpath)))
                    driver.find_element_by_xpath(xxpath).click()
                    time.sleep(2)
                    break
                except:
                    tries+=1   
    def hoverclick(xxpath):
        hover = ActionChains(driver).move_to_element(driver.find_element_by_xpath(xxpath))
        action = webdriver.common.action_chains.ActionChains(driver)

        tries=0
        while tries<3:
            try:
                hover.perform()
                break
            except:
                tries+=1
        time.sleep(2)
        
        action.click()
        action.perform()
        
    def randyzip(xxpath):
        action = webdriver.common.action_chains.ActionChains(driver)
        el=driver.find_element_by_xpath(xxpath)
        action.move_to_element_with_offset(el,420,200)
        action.click()
        action.perform()
        time.sleep(1)
        
    
    driver = webdriver.Chrome(executable_path=r'./chromedriver.exe', options=options)
    
    url="https://www.albertsons.com"
    driver.get(url)
    time.sleep(4)
    
    hoverclick("/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/div/div[2]/div[2]/div/div[2]/div[2]/button[3]/span/span[3]")
    
    #Change zip code
        
    
    hoverclick("/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input")
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input').send_keys(mlookup[market])
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input').send_keys(Keys.ENTER)

    randyzip('/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input')
    randyzip('/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input')

    
    summary=list()    
    
    
    brizand=("Karbach","Shiner","Deep Ellum","Blue Moon","Saint Arnold","Corona","Michelob Ultra","Truly","White Claw")
        
    for b in brizand:
        
        k=b+" beer"
        searchbar=driver.find_element_by_xpath("/html/body/div[2]/div/div/div[1]/div/div/div/div/div[3]/div[2]/div[6]/form/div/div/input")
        for L in range(20):
            searchbar.send_keys(Keys.DELETE)
            searchbar.send_keys(Keys.BACKSPACE)
        searchbar.send_keys(k)
        searchbar.send_keys(Keys.ENTER)
        time.sleep(5)
        
        c=0
        while c<10:
            ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            c+=1
        time.sleep(1)
        ActionChains(driver).send_keys(Keys.PAGE_UP).perform()
        
        #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        #time.sleep(5)
        #html_content = driver.execute_script('return document.body.innerHTML')
        
        page_source = driver.page_source
        #soup = BeautifulSoup(html_content, 'lxml')
        soup = BeautifulSoup(page_source)

        product= soup.body.find_all(class_='ab-lazy loaded')
        price_selector = soup.find_all(class_='product-price-con')
        
        
        price=[]
        brand=[]
        pkg=[]
        
        for pr,br in zip(price_selector,product):
                 
            try:
                brand.append(str(re.findall(rf'{b}.*? -',str(br))[0])[:-2])
                pkg.append(str(re.findall(r'- .*?(?:Oz|FZ)',str(br))[0][2:]))
                price.append(str(re.findall(r'Your Price.*?\$[0-9.]*',str(pr))[0][18:]))
            except:
                continue
        
        
        print([len(pkg),len(brand),len(price)])
        
        
        for i in range(len(pkg)):
            summary.append({"cdate":Current_Date,"store":"Albertsons Network","market":market,"pkg":pkg[i],"brand":brand[i],"price":price[i]})
        
        
    driver.close()       
    return summary
    






driver = webdriver.Chrome(executable_path=r'./chromedriver.exe', options=options)


cities=["Dallas","Houston"]

Combined = list()

for i in cities:
    Combined.append(randyscrape(i))
     


filename=Current_Date+"albert.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()


print("done")



In [ ]:
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

##Walmart
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime
import json

options = webdriver.ChromeOptions()
#options.add_argument('headless')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
 

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')

mlookup={"Dallas":"75001","San Angelo":"76901","Houston":"77001","San Antonio":"78232",
         "Austin":"73301","Corpus Christi":"78336","College Station":"77802","Waco":"76633",
         "Victoria":"77901","McAllen":"78501"}




def wmscrape(market):
    print(market)
    def clickit(xxpath):
            tries=0
            while tries<3:
                try:
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xxpath)))
                    driver.find_element_by_xpath(xxpath).click()
                    time.sleep(2)
                    break
                except:
                    tries+=1   
    def hoverclick(xxpath):
        hover = ActionChains(driver).move_to_element(driver.find_element_by_xpath(xxpath))
        action = webdriver.common.action_chains.ActionChains(driver)

        tries=0
        while tries<3:
            try:
                hover.perform()
                break
            except:
                tries+=1
        time.sleep(2)
        
        action.click()
        action.perform()
        
    def randyzip(xxpath):
        action = webdriver.common.action_chains.ActionChains(driver)
        el=driver.find_element_by_xpath(xxpath)
        action.move_to_element_with_offset(el,10,10)
        action.click()
        action.perform()
        time.sleep(1)
        
    
    login=market.lower()+"31415@gmail.com"
    pw=market.lower()+"31415!!"
    
    url="https://www.walmart.com"
    driver.get(url)
    time.sleep(4)
    
    
    
    #Change zip code
    hoverclick("/html/body/div/div[1]/div/div/div/section/div/button[2]")
    time.sleep(2)
    randyzip("/html/body/div[3]/div/div[3]/div[1]/div/div[2]/div/form/div/label/span")
    #randyzip("/html/body/div[3]/div/div[3]/div[1]/div/div[2]/div/section/form/div/div/input")
    time.sleep(2)
    
    actions = ActionChains(driver)
    actions.send_keys(Keys.DELETE)
    actions.send_keys(Keys.DELETE)
    actions.send_keys(Keys.DELETE)
    actions.send_keys(Keys.DELETE)
    actions.send_keys(Keys.DELETE)
    actions.send_keys(Keys.DELETE)
    
    
    actions.send_keys(mlookup[market])
    actions.perform()
    actions.send_keys(Keys.ENTER)
    actions.perform()
    
    #driver.find_element_by_xpath("/html/body/div[2]/div/div[3]/div[1]/div/div[2]/div/form/div/div/input").send_keys(Keys.BACKSPACE)
    #print("wtf")
    #driver.find_element_by_xpath("/html/body/div[3]/div/div[3]/div[1]/div/div[2]/div/form/div/div/input").send_keys(Keys.BACKSPACE)
    #driver.find_element_by_xpath("/html/body/div[3]/div/div[3]/div[1]/div/div[2]/div/form/div/div/input").send_keys(Keys.BACKSPACE)
    #driver.find_element_by_xpath("/html/body/div[3]/div/div[3]/div[1]/div/div[2]/div/form/div/div/input").send_keys(Keys.BACKSPACE)
    #driver.find_element_by_xpath("/html/body/div[3]/div/div[3]/div[1]/div/div[2]/div/form/div/div/input").send_keys(Keys.BACKSPACE)
    #driver.find_element_by_xpath("/html/body/div[3]/div/div[3]/div[1]/div/div[2]/div/form/div/div/input").send_keys(mlookup[market])
    #driver.find_element_by_xpath("/html/body/div[3]/div/div[3]/div[1]/div/div[2]/div/form/div/div/input").send_keys(Keys.ENTER)
    
    time.sleep(3)
    
    summary=list()    
    

    brizand=("Shiner","Karbach","Deep Ellum","Blue Moon","Saint Arnold","Corona","Michelob Ultra","Truly","White Claw")
        
    for b in brizand:
        
        k=b+" beer"
        searchbar=driver.find_element_by_xpath("/html/body/div/div[1]/div/span/header/div/form/div/input")
        if b != "Shiner":

            for L in range(60):
                searchbar.send_keys(Keys.DELETE)
                
            for L in range(60):
                
                searchbar.send_keys(Keys.BACKSPACE)
        
        searchbar.send_keys(k)
        searchbar.send_keys(Keys.ENTER)
        time.sleep(3)
        
        



        page_source = driver.page_source

        soup = BeautifulSoup(page_source)

        product=[]
        prices=[]
        
        wtflist=['w_Dr','w_DY','w_DP','w_Cj','w_D1']
        merged= soup.body.find_all(class_='w_D1')
        
        for m in merged:
            if b in str(m):
                product.append(m)
            elif "current price" in str(m):
                prices.append(m)
                
            

        price=[]
        brand=[]
        pkg=[]

        for i in range(len(product)):
            brand.append(str(re.search(r'\>.*?(,|\d)',str(product[i])).group(0))[1:-1])
            
            pkg.append(str(re.search(rf'\d.*?(pack|Pack|pk|c|B|z|b|C)',str(product[i])).group(0)))
            
            price.append(str(re.findall(rf'\$[0-9.]*',str(prices[i]))[0]))
            
            
        
        
        print([len(pkg),len(brand),len(price)])
        
        
        for i in range(len(pkg)):
            summary.append({"cdate":Current_Date,"store":"Walmart","market":market,"pkg":pkg[i],"brand":brand[i],"price":price[i]})
        




        driver.back()
        time.sleep(3)




    
    return summary


driver = webdriver.Chrome(executable_path=r'./chromedriver.exe', options=options)
           

#cities=["Dallas","San Angelo","Houston","San Antonio","Austin","Corpus Christi","College Station","Waco","Victoria","McAllen"]
cities=["Austin","Corpus Christi","College Station","Waco","Dallas","San Angelo","Houston","San Antonio","Victoria","McAllen"]
#
Combined = list()



for i in cities:
    while True:
        try:
            Combined.append(wmscrape(i))
            break
        except:
            print("Fail")
            driver.close()
            driver = webdriver.Chrome(executable_path=r'./chromedriver.exe', options=options)

driver.close()
    
filename=Current_Date+"wm.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()

print("done")